In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Import the necessary libraries

In [69]:
import pandas as pd
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from numpy import set_printoptions
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

### Import the dataset

In [70]:
os.chdir(r"C:\Users\achatter\Downloads\SDSC_Project")
flour_data = pd.read_csv('flour_dataset_cleaned.csv',skiprows=[0], usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13], names=["gluten_content","dough_elasticity","damp_time","pack_weight","ash_content","production_recipe","moisture_content","starch_content","production_mill"
                                                                                                                   ,"package_volume","protein_content","color","Quality"])

flour_data['profit'] = 0.0

for i in flour_data.index:
    if flour_data['Quality'][i] == 'Low':
        flour_data['profit'][i] = 1.2-1
        
    if flour_data['Quality'][i] == 'Average':
        flour_data['profit'][i] = 2.5-1
        
    if flour_data['Quality'][i] == 'High':
        flour_data['profit'][i] = 5-1
        
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'Production Recipe' column 
encoded_df = pd.DataFrame(encoder.fit_transform(flour_data[['production_recipe']]).toarray())

#merge one-hot encoded columns back with original DataFrame
flour_data_encoded = flour_data.join(encoded_df).drop(columns=['production_recipe'])

flour_data_encoded.rename(columns = {0:'recipe1',1:'recipe2',2:'recipe3',3:'recipe4',4:'recipe5'}, inplace = True)

#creating instance of one-hot-encoder
encoder1 = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'Production Recipe' column 
encoded_df = pd.DataFrame(encoder1.fit_transform(flour_data_encoded[['production_mill']]).toarray())

#merge one-hot encoded columns back with original DataFrame
flour_data_encoded = flour_data_encoded.join(encoded_df).drop(columns=['production_mill'])

flour_data_encoded.rename(columns = {0:'mill_AOM',1:'mill_HPW',2:'mill_ROW'}, inplace = True)
flour_data_full = flour_data_encoded

flour_data_full['quality_y'] = 0

for i in flour_data_full.index:
    if flour_data_full['Quality'][i] == 'Low':
        flour_data_full['quality_y'][i] = 0
        
    if flour_data_full['Quality'][i] == 'Average':
        flour_data_full['quality_y'][i] = 1
        
    if flour_data_full['Quality'][i] == 'High':
        flour_data_full['quality_y'][i] = 2

#creating instance of one-hot-encoder
encoder2 = OneHotEncoder(handle_unknown='ignore')
#perform one-hot encoding on 'Quality' column 
encoded_df = pd.DataFrame(encoder2.fit_transform(flour_data_encoded[['Quality']]).toarray())

#merge one-hot encoded columns back with original DataFrame
flour_data_encoded = flour_data_encoded.join(encoded_df).drop(columns=['Quality'])

flour_data_encoded.rename(columns = {0:'Quality_Average',1:'Quality_High',2:'Quality_Low'}, inplace = True)

C:\Users\achatter\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\achatter\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\achatter\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\achatter\AppData\Roaming\P

### Generate the correlation matrix for the variables

In [109]:
data_corr = flour_data_encoded.drop(columns=['quality_y','profit']).corr(method='pearson', min_periods=1)

for ir, row in enumerate(data_corr.index):
    for ic, col in enumerate(data_corr.columns):
        if ir >= ic:
            data_corr[row][col] = float("NaN")
            
fig_corr = px.imshow(data_corr,labels=dict(x="Features", y="Features", color="Correlation Coefficient"), width=1800, height=1200, template="simple_white")
fig_corr.update_layout(font=dict(family="Courier New, monospace",size=20))
fig_corr.update_xaxes(tickangle=270)
fig_corr.show()

### Plot for Gluten Content v/s Quality

In [72]:
fig_gc = px.histogram(flour_data, x="gluten_content", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_gc.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_gc.update_xaxes(title_text="<b>Gluten Content</b>(%)")
fig_gc.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_gc.show()

### Additional Profit potential from Gluten Content Optimisation

In [73]:
profit_less_gc = 0.0
profit_more_gc = 0.0
count_less = 0
count_more = 0

for i in flour_data.index:
    if flour_data['gluten_content'][i] < 11.4:
        profit_less_gc += flour_data['profit'][i]
        count_less += 1
        
    if flour_data['gluten_content'][i] >= 11.4:
        profit_more_gc += flour_data['profit'][i]
        count_more += 1
        
profit_less_gc = profit_less_gc/count_less
profit_more_gc = profit_more_gc/count_more

# More Average profit if choosing flour with Gluten Content < 11.4%
avg_profit_inc = abs(profit_less_gc-profit_more_gc)
        
print("Increase in profit from Gluten content optimization: "+str(round(avg_profit_inc, 2))+" CHF/package") 

Increase in profit from Gluten content optimization: 0.5 CHF/package


### Plot for Dough Elasticity Index v/s Quality

In [74]:
fig_ei = px.histogram(flour_data, x="dough_elasticity", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_ei.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_ei.update_xaxes(title_text="<b>Dough Elasticity Index</b>(%)")
fig_ei.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_ei.show()

### Additional Profit potential from Dough Elasticity Index Optimisation

In [75]:
profit_less_ei = 0.0
profit_more_ei = 0.0
count_less = 0
count_more = 0

for i in flour_data.index:
    if flour_data['dough_elasticity'][i] < 34:
        profit_less_ei += flour_data['profit'][i]
        count_less += 1
        
    if flour_data['dough_elasticity'][i] >= 34:
        profit_more_ei += flour_data['profit'][i]
        count_more += 1
        
profit_less_ei = profit_less_ei/count_less
profit_more_ei = profit_more_ei/count_more

# More Average profit if choosing flour with Dough Elasticity Index > 34%
avg_profit_inc = abs(profit_less_ei-profit_more_ei)
        
print("Increase in profit from Dough Elasticity Index optimization: "+str(round(avg_profit_inc, 2))+" CHF/package") 

Increase in profit from Dough Elasticity Index optimization: 0.44 CHF/package


### Plot for Dampening Time v/s Quality

In [76]:
fig_dt = px.histogram(flour_data, x="damp_time", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_dt.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_dt.update_xaxes(title_text="<b>Dampening Time</b>(hr.)")
fig_dt.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_dt.show()

### Additional Profit potential from Dampening Time Optimisation

### Plot for Package Weight v/s Quality

In [77]:
fig_pw = px.histogram(flour_data, x="pack_weight", color="Quality", color_discrete_sequence=['red','orange','green'], nbins=50)

fig_pw.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_pw.update_xaxes(title_text="<b>Package Weight</b>(g.)")
fig_pw.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_pw.show()

### Plot for Ash Content v/s Quality

In [78]:
fig_ac = px.histogram(flour_data, x="ash_content", color="Quality", color_discrete_sequence=['red','orange','green'], nbins=1000)

fig_ac.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_ac.update_xaxes(title_text="<b>Ash Content</b>(%)", range=[1.4, 2])
fig_ac.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_ac.show()

### Additional Profit potential from Ash Content Optimisation

In [110]:
profit_less_ac = 0.0
profit_more_ac = 0.0
count_less = 0
count_more = 0

for i in flour_data.index:
    if flour_data['ash_content'][i] < 1.8:
        profit_less_ac += flour_data['profit'][i]
        count_less += 1
        
    if flour_data['ash_content'][i] >= 1.8:
        profit_more_ac += flour_data['profit'][i]
        count_more += 1
        
profit_less_ac = profit_less_ac/count_less
profit_more_ac = profit_more_ac/count_more

# More Average profit if choosing flour with Ash Content < 1.8%
avg_profit_inc = abs(profit_less_ac-profit_more_ac)
        
print("Increase in profit from Ash content optimization: "+str(round(avg_profit_inc, 2))+" CHF/package") 

Increase in profit from Ash content optimization: 0.73 CHF/package


### Plot for Production Recipe v/s Quality

In [80]:
fig_pq = px.histogram(flour_data, x="production_recipe", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_pq.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_pq.update_xaxes(title_text="<b>Production Recipe</b>(-)")
fig_pq.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_pq.show()

### Additional Profit potential from Recipe Optimisation

In [81]:
profit_less_pr = 0.0
profit_more_pr = 0.0
count_less = 0
count_more = 0

for i in flour_data.index:
    if flour_data['production_recipe'][i] == 3:
        profit_less_pr += flour_data['profit'][i]
        count_less += 1
        
    if flour_data['production_recipe'][i] == 5:
        profit_more_pr += flour_data['profit'][i]
        count_more += 1
        
profit_less_pr = profit_less_pr/count_less
profit_more_pr = profit_more_pr/count_more

# More Average profit if choosing flour with Recipe 3 over Recipe 5
avg_profit_inc = abs(profit_less_pr-profit_more_pr)
        
print("Increase in profit by choosing Recipe 3 instead of 5: "+str(round(avg_profit_inc, 2))+" CHF/package") 

Increase in profit by choosing Recipe 3 instead of 5: 0.13 CHF/package


### Plot for Moisture Content v/s Quality

In [82]:
fig_mc = px.histogram(flour_data, x="moisture_content", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_mc.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_mc.update_xaxes(title_text="<b>Moisture Content</b>(%)", range=[13.8, 15.4])
fig_mc.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_mc.show()

### Additional Profit potential from Moisture Content Optimisation

In [83]:
profit_less_mc = 0.0
profit_more_mc = 0.0
count_less = 0
count_more = 0

for i in flour_data.index:
    if flour_data['moisture_content'][i] < 14.4:
        profit_less_mc += flour_data['profit'][i]
        count_less += 1
        
    if flour_data['moisture_content'][i] >= 14.4:
        profit_more_mc += flour_data['profit'][i]
        count_more += 1
        
profit_less_mc = profit_less_mc/count_less
profit_more_mc = profit_more_mc/count_more

# More Average profit if choosing flour with Moisture Content < 14.4%
avg_profit_inc = abs(profit_less_mc-profit_more_mc)
        
print("Increase in profit by Moisture content optimization: "+str(round(avg_profit_inc, 2))+" CHF/package") 

Increase in profit by Moisture content optimization: 0.42 CHF/package


### Plot for Starch Content v/s Quality

In [84]:
fig_sc = px.histogram(flour_data, x="starch_content", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_sc.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_sc.update_xaxes(title_text="<b>Starch Content</b>(%)", range=[48, 63])
fig_sc.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_sc.show()

### Additional Profit potential from Starch Content Optimisation

In [85]:
profit_less_sc = 0.0
profit_more_sc = 0.0
count_less = 0
count_more = 0

for i in flour_data.index:
    if flour_data['starch_content'][i] < 55:
        profit_less_sc += flour_data['profit'][i]
        count_less += 1
        
    if flour_data['starch_content'][i] >= 55:
        profit_more_sc += flour_data['profit'][i]
        count_more += 1
        
profit_less_sc = profit_less_sc/count_less
profit_more_sc = profit_more_sc/count_more

# More Average profit if choosing flour with Starch Content > 55%
avg_profit_inc = abs(profit_less_sc-profit_more_sc)
        
print("Increase in profit by Starch content optimization: "+str(round(avg_profit_inc, 2))+" CHF/package") 

Increase in profit by Starch content optimization: 0.44 CHF/package


### Plot for Production Mill v/s Quality

In [86]:
fig_pm = px.histogram(flour_data, x="production_mill", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_pm.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_pm.update_xaxes(title_text="<b>Production Mill</b>(-)")
fig_pm.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_pm.show()

### Additional Profit potential from Production Mill Optimisation

In [87]:
profit_less_pm = 0.0
profit_more_pm = 0.0
count_less = 0
count_more = 0

for i in flour_data.index:
    if flour_data['production_mill'][i] == "Happy Blackberry Windmill":
        profit_less_pm += flour_data['profit'][i]
        count_less += 1
        
    if flour_data['production_mill'][i] == "Ancient Oak Mill":
        profit_more_pm += flour_data['profit'][i]
        count_more += 1
        
profit_less_pm = profit_less_pm/count_less
profit_more_pm = profit_more_pm/count_more

# More Average profit if choosing flour with Recipe 3 over Recipe 5
avg_profit_inc = abs(profit_less_pm-profit_more_pm)
        
print("Increase in profit by choosing Ancient Oak Mill instead of Happy Blackberry Windmill: "+str(round(avg_profit_inc, 2))+" CHF/package")
profit_more_pm

Increase in profit by choosing Ancient Oak Mill instead of Happy Blackberry Windmill: 2.37 CHF/package


3.366666666666667

### Plot for Package Volume v/s Quality

In [88]:
fig_pv = px.histogram(flour_data, x="package_volume", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_pv.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_pv.update_xaxes(title_text="<b>Package Volume</b>(cm3)")
fig_pv.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_pv.show()

### Plot for Proteins Content v/s Quality

In [89]:
fig_pc = px.histogram(flour_data, x="protein_content", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_pc.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_pc.update_xaxes(title_text="<b>Protein Content</b>(%)", range=[7, 14])
fig_pc.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_pc.show()

### Additional Profit potential from Protein Content Optimisation

In [90]:
profit_less_pc = 0.0
profit_more_pc = 0.0
count_less = 0
count_more = 0

for i in flour_data.index:
    if flour_data['protein_content'][i] < 11.5:
        profit_less_pc += flour_data['profit'][i]
        count_less += 1
        
    if flour_data['ash_content'][i] >= 11.5:
        profit_more_pc += flour_data['profit'][i]
        count_more += 1
        
profit_less_pc = profit_less_pc/count_less
profit_more_pc = profit_more_pc/count_more

# More Average profit if choosing flour with Protein Content < 11.5%
avg_profit_inc = abs(profit_less_pc-profit_more_pc)
        
print("Increase in profit from Protein content optimization: "+str(round(avg_profit_inc, 2))+" CHF/package") 

Increase in profit from Protein content optimization: 0.9 CHF/package


### Plot for Color Tone v/s Quality

In [91]:
fig_c = px.histogram(flour_data, x="color", color="Quality", color_discrete_sequence=['red','orange','green'])

fig_c.update_yaxes(title_text="<b>No. of flour samples</b>(-)")
fig_c.update_xaxes(title_text="<b>Color</b>(lighter -> darker)")
fig_c.update_layout(barmode='group',template='plotly_white', width=1500, height=800, font=dict(family="Courier New, monospace",size=20), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig_c.show()

### Training the model using KNN Classifiers

In [117]:
# Importing the dataset
dataset = flour_data_full
X = dataset.drop(columns=['Quality','quality_y','profit']).values
y = dataset['quality_y'].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Training the K-NN model on the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 9, algorithm='auto', metric = 'minkowski', p = 1)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", round(accuracy_score(y_test, y_pred),3))

# Plot confusion Matrix
fig = px.imshow(cm, labels=dict(x="Predicted Labels", y="Actual Labels", color="Flour Samples"), x=['Low', 'Average', 'High'], y=['Low', 'Average', 'High'], text_auto=True, color_continuous_scale='brwnyl')
fig.update_xaxes(side="top")
fig.update_layout(template='plotly_white', width=900, height=900, font=dict(family="Courier New, monospace",size=20))
fig.show()

[[ 60  36   0]
 [  8 179   0]
 [  2   0   3]]
Accuracy: 0.84


### Grid Search for best parameters with KNN Classifiers

In [93]:
# Importing the dataset
dataset = flour_data_full
X = dataset.drop(columns=['Quality','quality_y','profit']).values
y = dataset['quality_y'].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Training the Random Forest Classification model on the Training set
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(knn,parameters,cv=5,scoring='accuracy')
cv.fit(X_train, y_train)

def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
display(cv)

Best parameters are: {'algorithm': 'auto', 'n_neighbors': 9, 'p': 1}


0.602 + or -0.02 for the {'algorithm': 'auto', 'n_neighbors': 1, 'p': 1}
0.605 + or -0.022 for the {'algorithm': 'auto', 'n_neighbors': 1, 'p': 2}
0.579 + or -0.019 for the {'algorithm': 'auto', 'n_neighbors': 2, 'p': 1}
0.562 + or -0.024 for the {'algorithm': 'auto', 'n_neighbors': 2, 'p': 2}
0.635 + or -0.031 for the {'algorithm': 'auto', 'n_neighbors': 3, 'p': 1}
0.608 + or -0.023 for the {'algorithm': 'auto', 'n_neighbors': 3, 'p': 2}
0.609 + or -0.034 for the {'algorithm': 'auto', 'n_neighbors': 4, 'p': 1}
0.589 + or -0.017 for the {'algorithm': 'auto', 'n_neighbors': 4, 'p': 2}
0.626 + or -0.022 for the {'algorithm': 'auto', 'n_neighbors': 5, 'p': 1}
0.619 + or -0.016 for the {'algorithm': 'auto', 'n_neighbors': 5, 'p': 2}
0.615 + or -0.028 for the {'algorithm': 'auto', 'n_neighbors': 6, 'p': 1}
0.605 + or -0.025 for the {'algorithm': 'auto', 'n_neighbors': 6, 'p': 2}
0.629 + or -0.021 for the {'algorithm': 'a

### Training the model using Naive Bayes Classifiers

In [94]:
# Importing the dataset
dataset = flour_data_full
X = dataset.drop(columns=['Quality','quality_y','profit']).values
y = dataset['quality_y'].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Training the Naive Bayes model on the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB(var_smoothing=5e-05)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", round(accuracy_score(y_test, y_pred),3))

# Plot confusion Matrix
fig = px.imshow(cm, labels=dict(x="Predicted Labels", y="Actual Labels", color="Flour Samples"), x=['Low', 'Average', 'High'], y=['Low', 'Average', 'High'], text_auto=True, color_continuous_scale='brwnyl')
fig.update_xaxes(side="top")
fig.update_layout(template='plotly_white', width=900, height=900, font=dict(family="Courier New, monospace",size=20))
fig.show()

[[ 30  66   0]
 [  1 186   0]
 [  1   1   3]]
Accuracy: 0.76


### Grid Search for best parameters with Naive Bayes Classifiers

In [95]:
# Importing the dataset
dataset = flour_data_full
X = dataset.drop(columns=['Quality','quality_y','profit']).values
y = dataset['quality_y'].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Training the Random Forest Classification model on the Training set
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
parameters = {'var_smoothing': np.logspace(0,-9, num=100)}

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(gnb,parameters,cv=5,scoring='accuracy')
cv.fit(X_train, y_train)

def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
display(cv)

Best parameters are: {'var_smoothing': 6.579332246575683e-06}


0.653 + or -0.024 for the {'var_smoothing': 1.0}
0.644 + or -0.018 for the {'var_smoothing': 0.8111308307896871}
0.647 + or -0.023 for the {'var_smoothing': 0.657933224657568}
0.654 + or -0.028 for the {'var_smoothing': 0.533669923120631}
0.657 + or -0.029 for the {'var_smoothing': 0.43287612810830584}
0.659 + or -0.031 for the {'var_smoothing': 0.3511191734215131}
0.662 + or -0.031 for the {'var_smoothing': 0.2848035868435802}
0.669 + or -0.029 for the {'var_smoothing': 0.23101297000831597}
0.669 + or -0.028 for the {'var_smoothing': 0.1873817422860384}
0.67 + or -0.03 for the {'var_smoothing': 0.15199110829529336}
0.671 + or -0.027 for the {'var_smoothing': 0.12328467394420659}
0.677 + or -0.029 for the {'var_smoothing': 0.1}
0.677 + or -0.027 for the {'var_smoothing': 0.08111308307896872}
0.676 + or -0.027 for the {'var_smoothing': 0.0657933224657568}
0.676 + or -0.028 for the {'var_smoothing': 0.0533669923120631}
0.676

### Training the model using Decision Tree Classifiers

In [119]:
# Importing the dataset
dataset = flour_data_full
X = dataset.drop(columns=['Quality','quality_y','profit']).values
y = dataset['quality_y'].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Training the Decision Tree Classification model on the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'gini',splitter='best',max_depth=None,min_samples_split=10,min_samples_leaf=1,max_features='auto',random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", round(accuracy_score(y_test, y_pred),3))

# Plot confusion Matrix
fig = px.imshow(cm, labels=dict(x="Predicted Labels", y="Actual Labels", color="Flour Samples"), x=['Low', 'Average', 'High'], y=['Low', 'Average', 'High'], text_auto=True, color_continuous_scale='brwnyl')
fig.update_xaxes(side="top")
fig.update_layout(template='plotly_white', width=900, height=900, font=dict(family="Courier New, monospace",size=20))
fig.show()

[[ 78  18   0]
 [ 33 154   0]
 [  2   1   2]]
Accuracy: 0.812


### Grid Search for best parameters with Decision Tree Classifier

In [97]:
# Importing the dataset
dataset = flour_data_full
X = dataset.drop(columns=['Quality','quality_y','profit']).values
y = dataset['quality_y'].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Training the Random Forest Classification model on the Training set
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2,4,8,16,32,None],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(tree,parameters,cv=5,scoring = 'accuracy')
cv.fit(X_train, y_train)

def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
display(cv)

Best parameters are: {'criterion': 'gini', 'max_depth': 4, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 5, 'splitter': 'best'}


0.709 + or -0.058 for the {'criterion': 'gini', 'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
0.641 + or -0.06 for the {'criterion': 'gini', 'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'}
0.684 + or -0.051 for the {'criterion': 'gini', 'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'best'}
0.608 + or -0.069 for the {'criterion': 'gini', 'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'random'}
0.678 + or -0.07 for the {'criterion': 'gini', 'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'best'}
0.632 + or -0.076 for the {'criterion': 'gini', 'max_depth': 2, 'max_f

### Training the model using Random Forest Classifiers

In [98]:
# Importing the dataset
dataset = flour_data_full
X = dataset.drop(columns=['Quality','quality_y','profit']).values
y = dataset['quality_y'].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0, max_depth=None)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", round(accuracy_score(y_test, y_pred),3))

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

# Plot confusion Matrix
fig = px.imshow(cm, labels=dict(x="Predicted Labels", y="Actual Labels", color="Flour Samples"), x=['Low', 'Average', 'High'], y=['Low', 'Average', 'High'], text_auto=True, color_continuous_scale='brwnyl')
fig.update_xaxes(side="top")
fig.update_layout(template='plotly_white', width=900, height=900, font=dict(family="Courier New, monospace",size=20))
fig.show()

[[ 74  22   0]
 [ 13 174   0]
 [  2   0   3]]
Accuracy: 0.872
Accuracy: 83.12 %
Standard Deviation: 2.14 %


### Grid Search for best parameters with Random Forest Classifier

In [99]:
# Importing the dataset
dataset = flour_data_full
X = dataset.drop(columns=['Quality','quality_y','profit']).values
y = dataset['quality_y'].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
parameters = {"n_estimators":[5,10,50,100,250],"max_depth":[2,4,8,16,32,None],"criterion":['gini', 'entropy']}

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rfc,parameters,cv=5,scoring = 'accuracy')
cv.fit(X_train, y_train)

def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
display(cv)

Best parameters are: {'criterion': 'entropy', 'max_depth': 8, 'n_estimators': 100}


0.751 + or -0.025 for the {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 5}
0.765 + or -0.031 for the {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 10}
0.78 + or -0.019 for the {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 50}
0.782 + or -0.017 for the {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 100}
0.788 + or -0.015 for the {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 250}
0.773 + or -0.02 for the {'criterion': 'gini', 'max_depth': 4, 'n_estimators': 5}
0.803 + or -0.013 for the {'criterion': 'gini', 'max_depth': 4, 'n_estimators': 10}
0.827 + or -0.013 for the {'criterion': 'gini', 'max_depth': 4, 'n_estimators': 50}
0.831 + or -0.009 for the {'criterion': 'gini', 'max_depth': 4, 'n_estimators': 100}
0.825 + or -0.008 for the {'criterion': 'gini', 'max_depth': 4, 'n_estimators': 250}
0.802 + or -0.015 for the {'criterion': 'gini', 'max_depth': 8, 'n_estimato

### Training the model using Artificial Neural Networks

In [100]:
# Importing the dataset
dataset = flour_data_full
X = dataset.drop(columns=['Quality','quality_y','profit']).values
y = dataset['quality_y'].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Part 2 - Building the ANN

# Initializing the ANN
ann = tf.keras.models.Sequential()

# Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Adding the third hidden layer
ann.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Adding the third hidden layer
ann.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Adding the output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the ANN

# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the ANN on the Training set
ann.fit(X_train, y_train, batch_size = 32, epochs = 200)

# Part 4 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = np.around(ann.predict(X_test),0)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", round(accuracy_score(y_test, y_pred),3))

Train on 1077 samples
Epoch 1/200
1077/1077 [==============================] - 1s 703us/sample - loss: 2.2095 - accuracy: 0.5070
Epoch 2/200
1077/1077 [==============================] - 0s 74us/sample - loss: 1.1012 - accuracy: 0.5200
Epoch 3/200
1077/1077 [==============================] - 0s 79us/sample - loss: 0.6878 - accuracy: 0.5850
Epoch 4/200
1077/1077 [==============================] - 0s 79us/sample - loss: 0.6868 - accuracy: 0.6082
Epoch 5/200
1077/1077 [==============================] - 0s 76us/sample - loss: 0.8969 - accuracy: 0.5478
Epoch 6/200
1077/1077 [==============================] - 0s 81us/sample - loss: 1.0482 - accuracy: 0.5515
Epoch 7/200
1077/1077 [==============================] - 0s 75us/sample - loss: 0.7130 - accuracy: 0.5980
Epoch 8/200
1077/1077 [==============================] - 0s 75us/sample - loss: 0.9808 - accuracy: 0.5543
Epoch 9/200
1077/1077 [==============================] - 0s 75us/sample - loss: 0.6804 - accuracy: 0.5989
Epoch 10/200
1077/1077 

1077/1077 [==============================] - 0s 78us/sample - loss: 0.5974 - accuracy: 0.6630
Epoch 78/200
1077/1077 [==============================] - 0s 82us/sample - loss: 0.6085 - accuracy: 0.6620
Epoch 79/200
1077/1077 [==============================] - 0s 76us/sample - loss: 0.6272 - accuracy: 0.6388
Epoch 80/200
1077/1077 [==============================] - 0s 78us/sample - loss: 0.6066 - accuracy: 0.6630
Epoch 81/200
1077/1077 [==============================] - 0s 73us/sample - loss: 0.6257 - accuracy: 0.6379
Epoch 82/200
1077/1077 [==============================] - 0s 75us/sample - loss: 0.6112 - accuracy: 0.6555
Epoch 83/200
1077/1077 [==============================] - 0s 71us/sample - loss: 0.6116 - accuracy: 0.6360
Epoch 84/200
1077/1077 [==============================] - 0s 73us/sample - loss: 0.6150 - accuracy: 0.6370
Epoch 85/200
1077/1077 [==============================] - 0s 71us/sample - loss: 0.6834 - accuracy: 0.5998
Epoch 86/200
1077/1077 [==========================

1077/1077 [==============================] - 0s 74us/sample - loss: 0.6009 - accuracy: 0.6611
Epoch 153/200
1077/1077 [==============================] - 0s 72us/sample - loss: 0.5897 - accuracy: 0.6630
Epoch 154/200
1077/1077 [==============================] - 0s 78us/sample - loss: 0.6130 - accuracy: 0.6472
Epoch 155/200
1077/1077 [==============================] - 0s 69us/sample - loss: 0.5887 - accuracy: 0.6657
Epoch 156/200
1077/1077 [==============================] - 0s 65us/sample - loss: 0.5940 - accuracy: 0.6620
Epoch 157/200
1077/1077 [==============================] - 0s 66us/sample - loss: 0.5950 - accuracy: 0.6565
Epoch 158/200
1077/1077 [==============================] - 0s 66us/sample - loss: 0.5926 - accuracy: 0.6611
Epoch 159/200
1077/1077 [==============================] - 0s 67us/sample - loss: 0.6109 - accuracy: 0.6509
Epoch 160/200
1077/1077 [==============================] - 0s 71us/sample - loss: 0.5960 - accuracy: 0.6639
Epoch 161/200
1077/1077 [=================

[[ 34  88   0]
 [  0 231   0]
 [  1   6   0]]
Accuracy: 0.736


In [124]:
algo = ['K-Nearest Neighbours','Decision Tree','Random Forest']
accu = [84.4,81.2,87.2]
colors = ['orange',] * 3
colors[2] = 'green'

fig = go.Figure([go.Bar(x=algo, y=accu, marker_color=colors)])
                 
fig.update_yaxes(title_text="<b>Accuracy</b>(%)")
fig.update_xaxes(title_text="<b>Classification Algorithm</b>")
fig.update_layout(template='plotly_white', width=1000, height=800, font=dict(family="Courier New, monospace",size=20))

fig.show()

### Calculate confidence Interval of profit by choosing AOM

In [125]:
import numpy as np 
from scipy.stats import t

x = np.array([4,0.2,4,4,4,4,4,4,0.2,4,4,4])

m = x.mean() 
s = x.std() 
dof = len(x)-1 
c95 = 0.95
c99 = 0.99
c999 = 0.9999

t_crit_95 = np.abs(t.ppf((1-c95)/2,dof))
ci_l_95 = m-s*t_crit_95/np.sqrt(len(x))
ci_u_95 = m+s*t_crit_95/np.sqrt(len(x))

t_crit_99 = np.abs(t.ppf((1-c99)/2,dof))
ci_l_99 = m-s*t_crit_99/np.sqrt(len(x))
ci_u_99 = m+s*t_crit_99/np.sqrt(len(x))

t_crit_999 = np.abs(t.ppf((1-c999)/2,dof))
ci_l_999 = m-s*t_crit_999/np.sqrt(len(x))
ci_u_999 = m+s*t_crit_999/np.sqrt(len(x))

# Confidence Interval of profit for 95% confidence
print("Confidence Interval of profit for 95% confidence: ("+str(round(ci_l_95,3))+", "+str(round(ci_u_95,3))+")")

# Confidence Interval of profit for 99% confidence
print("Confidence Interval of profit for 99% confidence: ("+str(round(ci_l_99,3))+", "+str(round(ci_u_99,3))+")")

# Confidence Interval of profit for 99.99% confidence
print("Confidence Interval of profit for 99.99% confidence: ("+str(round(ci_l_999,3))+", "+str(round(ci_u_999,3))+")")

Confidence Interval of profit for 95% confidence: (2.467, 4.266)
Confidence Interval of profit for 99% confidence: (2.097, 4.636)
Confidence Interval of profit for 99.99% confidence: (0.946, 5.787)


In [112]:
x

[4, 0.2, 4, 4, 4, 4, 4, 4, 0.2, 4, 4, 4]